<a href="https://colab.research.google.com/github/Jo-won/PatternRecognition_Term_2019Fall/blob/master/LevelPyramid_strongFeature_v2_word200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Downgrade cv
!yes | pip uninstall opencv-python
!yes | pip uninstall opencv-contrib-python
!yes | pip install opencv-python==3.4.2.16
!yes | pip install opencv-contrib-python==3.4.2.16
!pip install kmc2
! pip install cupy-cuda100

#mount
from google.colab import drive
drive.mount('/content/drive')

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 1.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 64.6MB/s 
     |████████████████████████████████| 102kB 3.8MB/s 
     |████████████████████████████████| 163kB 15.8MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252215 sha256=701d84eab8c66c04e27980a37847c45f4c932afafa485ce2086d4bb664e0ef42
  Stored in directory: /root/.cache/

In [0]:
# Kaggle API Renewal. It should be version 1.5.6

!ls -lha kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle -v


ls: cannot access 'kaggle.json': No such file or directory
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=7166bf6a438b91727d1574c2870677b588f696704a18abcfa0aad1cbe0ea0795
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", li

In [0]:
import os
import glob
import cv2
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC, SVC
from sklearn.cluster import KMeans
import copy
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import pandas as pd
import kmc2
from sklearn.cluster import MiniBatchKMeans


In [0]:
! ls
stepSize = 8
train_background = False
use_precompute = True
datasetPath = "/content/drive/My Drive/dataset/2019-ml-finalproject/"
datasetTrainPath = os.path.join(datasetPath + 'train')
datasetTestPath = os.path.join(datasetPath + 'testAll_v2')
className = os.listdir(datasetTrainPath)
className.sort()
classNameDict = dict(zip(range(0,len(className)), className))
print(className)

drive  sample_data
['BACKGROUND_Google', 'Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea_h

In [0]:
# load training dataset

trainDataset = []
trainLabel = []

lab = -1
for i, name in tqdm.tqdm(enumerate(className)):

    if className[i-1] != className[i]:
        lab+=1
    filename = os.listdir(os.path.join(datasetTrainPath+'/'+name))
    
    if train_background is False: # except background
    
        if i==0:
            continue
    
    for img in filename:

        if use_precompute is False:
            data = cv2.resize(cv2.imread(datasetTrainPath+'/'+name+'/'+img), (256,256), interpolation = cv2.INTER_LINEAR)
            data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
            trainDataset.append(cv2.GaussianBlur(data,(3,3),(stepSize/3)**2-0.25))

        trainLabel.append(lab)

if use_precompute is False:

    trainDataset = np.asarray(trainDataset)
    print(trainDataset.shape)

trainLabel = np.asarray(trainLabel)
print(trainLabel.shape)

if use_precompute is False:
    np.savez('/content/drive/My Drive/Colab Notebooks/pattern/repo/dataset/datatrainG', trainDataset=trainDataset, trainLabel=trainLabel)
else:
    savez_load = np.load('/content/drive/My Drive/Colab Notebooks/pattern/repo/dataset/datatrainG.npz')
    trainDataset = savez_load['trainDataset']
    trainLabel = savez_load['trainLabel']

102it [00:14,  6.91it/s]


(3030,)


In [0]:
# load test dataset

testDataset = []
filename = os.listdir(os.path.join(datasetTestPath))
filename.sort()

for img in tqdm.tqdm(filename):

    if use_precompute is False:
        data = cv2.resize(cv2.imread(datasetTestPath+'/'+img), (256,256), interpolation = cv2.INTER_LINEAR)
        data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
        testDataset.append(cv2.GaussianBlur(data,(3,3),(stepSize/3)**2-0.25))

if use_precompute is False:
    testDataset = np.asarray(testDataset)
    print(testDataset.shape)

if use_precompute is False:
    np.savez('/content/drive/My Drive/Colab Notebooks/pattern/repo/dataset/datatestG', testDataset=testDataset)
else:
    savez_load = np.load('/content/drive/My Drive/Colab Notebooks/pattern/repo/dataset/datatestG.npz')
    testDataset = savez_load['testDataset']


100%|██████████| 1692/1692 [00:00<00:00, 1421057.74it/s]


In [0]:
# compute dense SIFT 
def computeSIFT_(data,step_size):
    x = []
    for i in tqdm.tqdm(range(0, len(data)), desc = "Processing"):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]


        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        if np.linalg.norm(np.array(dense_feat[1]))>1.0:
            normalizeFeat = np.array(dense_feat[1])/float(np.linalg.norm(np.array(dense_feat[1])))
        x.append(normalizeFeat)
        
    return x

In [0]:
feats_save_path = '/content/drive/My Drive/Colab Notebooks/pattern/repo/pyramid/'

if os.path.isfile(feats_save_path + 'feats_save_G.npz'):
    savez_load = np.load(feats_save_path + 'feats_save_G.npz')
    trainFeats = savez_load['train']
    testFeats = savez_load['test']
    
else:
    trainFeats = computeSIFT_(trainDataset, stepSize)
    testFeats = computeSIFT_(testDataset, stepSize)
    np.savez(feats_save_path + 'feats_save_G', train=trainFeats, test=testFeats)


In [0]:
from joblib import dump, load
if os.path.isfile(feats_save_path + 'codeword_kmeans_G.npz') and os.path.isfile(feats_save_path + 'kmeans_G.joblib'):
    savez_load = np.load(feats_save_path + 'codeword_kmeans_G.npz')
    codebook = savez_load['codeword']
    codebooksize = savez_load['codebooksize']
    kmeans = load(feats_save_path + 'kmeans_G.joblib') 
else:
    codebooksize = 600
    seeding = kmc2.kmc2(np.array(trainFeats).reshape(-1,128), codebooksize)
    kmeans = KMeans(codebooksize, init=seeding).fit(np.array(trainFeats).reshape(-1,128))
    codebook = kmeans.cluster_centers_

    codeword_save_path = "/content/drive/My Drive/Colab Notebooks/pattern/repo/pyramid/"
    print(codebook.shape)
    print(codebook)
    dump(kmeans, codeword_save_path + 'kmeans_G.joblib') 
    np.savez(codeword_save_path + 'codeword_kmeans_G', codeword=codebook, codebooksize=codebooksize)

In [0]:
import math


def getImageFeaturesSPM(L, desc, kmeans, k):
    W = desc.shape[1]
    H = desc.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):             
                
                curr_desc = np.reshape(desc[y:y+h_step, x:x+w_step],(-1,128))
                predict = kmeans.predict(curr_desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            
            y = y + h_step
    
    hist = np.array(h).ravel()
    # normalize hist
    #dev = np.std(hist)
    #hist -= np.mean(hist)
    #hist /= dev
    return hist

def getHistogramSPM(L, data, kmeans, k):    
    x = []

    for i in tqdm.tqdm(range(len(data))):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

if os.path.isfile(feats_save_path + 'v2/pyramid_histo_G.npz'):
    savez_load = np.load(feats_save_path + 'v2/pyramid_histo_G.npz')
    train_histo = savez_load['train_histo']
    test_histo = savez_load['test_histo']
else:
    train_histo = getHistogramSPM(2, np.reshape(trainFeats,(-1,32,32,128)), kmeans, codebooksize)
    test_histo = getHistogramSPM(2, np.reshape(testFeats,(-1,32,32,128)), kmeans, codebooksize)
    np.savez(feats_save_path + 'v2/pyramid_histo_G', train_histo=train_histo, test_histo=test_histo)

In [0]:
# normalize histograms
scaler = preprocessing.StandardScaler().fit(train_histo)
train_histo_ = scaler.transform(train_histo)
test_histo_ = scaler.transform(test_histo)

In [0]:
print(train_histo.shape)
print(test_histo.shape)

(3030, 4200)
(1692, 4200)


In [0]:
def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in tqdm.tqdm(range(m)):
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result
if os.path.isfile(feats_save_path + 'v2/gramMatrix__G.npz'):
    savez_load = np.load(feats_save_path + 'v2/gramMatrix__G.npz')
    trainGramMatrix = savez_load['trainGramMatrix']
    testGramMatrix = savez_load['testGramMatrix']
else:
    trainGramMatrix = histogramIntersection(train_histo_, train_histo_)
    testGramMatrix = histogramIntersection(test_histo_, train_histo_)
    np.savez(feats_save_path + 'v2/gramMatrix__G', trainGramMatrix=trainGramMatrix, testGramMatrix=testGramMatrix)

In [0]:
print(trainGramMatrix.shape)
print(testGramMatrix.shape)

(3030, 3030)
(1692, 3030)


In [0]:
model = SVC(random_state=0, kernel = 'precomputed') #
model.fit(trainGramMatrix, trainLabel)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='precomputed', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
train = model.predict(trainGramMatrix)
print ("Accuracy:", np.mean(train == trainLabel)*100, "%")

Accuracy: 100.0 %


In [0]:
predict = model.predict(testGramMatrix)

In [0]:
df = pd.DataFrame({"Id":filename,"Category":predict})
df.index = np.arange(1,len(df)+1)
df.to_csv('/content/drive/My Drive/Colab Notebooks/pattern/repo/pyramid/buffer/results_191213_word200.csv',index=False, header=True)